In [1]:
import pip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error

Using TensorFlow backend.


In [2]:
CUDA_VISIBLE_DEVICES=""

In [30]:
song_app = pd.read_csv("../../SongApp.csv",header=-1,)

In [4]:
song_app.head()

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,NaN,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,0.0,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,1.0,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2.0,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,3.0,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [5]:
song_app.size

42661872

In [6]:
song_app.shape

(463716, 92)

In [31]:
# clean data
df_song = song_app.dropna()
df_song.__delitem__(0)

In [18]:
df_song.head()

,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,91
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
5,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [32]:
labels_column = df_song.iloc[0:,0]
labels = df_song.values[0:,0]
labels

array([ 2001.,  2001.,  2001., ...,  2006.,  2007.,  2007.])

In [33]:
df_song.__delitem__(1)

In [34]:
df_song['labels'] = labels_column

/home/phoenix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
num_columns = df_song.columns.shape[0]
columns = list(range(num_columns - 1))
columns_name = list(map(str,columns)) + ['labels']
df_song.columns = columns_name
df_song.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,labels
1,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,...,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061,2001
2,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,...,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345,2001
3,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,...,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382,2001
4,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,...,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903,2001
5,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,-4.69088,...,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263,2001


In [36]:
x_data = df_song.sample(150000)
y_data = x_data.values[0:,90]
y_data

array([ 2000.,  1991.,  1978., ...,  1997.,  1999.,  2005.])

In [37]:
x_data.__delitem__('labels')
x_data.head()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
16945,41.70658,42.90256,-2.26557,29.78797,7.74042,-18.36631,20.62528,6.18327,5.07653,12.36204,...,23.91205,-150.35553,-143.09115,-14.95596,7.65288,71.78503,115.31904,18.78442,112.27349,8.36616
50095,34.05327,-53.89563,-23.20444,-21.72451,6.50841,5.55310,-14.72488,5.63905,-10.79212,-2.55354,...,21.12820,-244.55795,4.40059,130.30515,-19.59308,80.84956,69.72612,-12.20763,-96.79340,12.53101
280789,37.15543,-31.76425,26.92129,-8.07270,-39.24462,-16.14830,-6.20393,-4.67406,11.87542,5.06061,...,26.85943,64.78609,31.37943,-19.90895,-4.26763,-26.77650,-9.11685,25.08648,280.19221,68.99568
439594,44.10014,36.15075,-16.79924,27.64898,-15.13887,10.73163,-28.81600,-5.35702,-7.74669,12.78935,...,32.29727,119.81648,-205.61828,-54.62186,18.82867,309.34501,68.01242,24.33625,-127.83227,-0.61660
155047,48.28101,-10.43531,-20.30768,-7.70055,-4.35982,-9.15873,-3.00551,-8.33774,1.73655,-7.27257,...,-48.25100,160.05463,48.69406,77.48567,26.68333,26.85367,-1.94171,4.14454,-57.12777,-15.06047


In [38]:
# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(135000, 90)
(15000, 90)
(135000,)
(15000,)


In [39]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(output_dim=1, activation='relu'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae', 'mape', 'cosine'])
    return model
X_train.shape

(135000, 90)

In [40]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=500, batch_size=X_train.shape[1], verbose=2)

In [41]:
kfold = KFold(n_splits=100, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/home/phoenix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`
  


Epoch 1/1
 - 24s - loss: 149504.1793 - mean_squared_error: 149504.1793 - mean_absolute_error: 264.2478 - mean_absolute_percentage_error: 13.2232 - cosine_proximity: -9.9840e-01
Epoch 1/1
 - 25s - loss: 144321.1552 - mean_squared_error: 144321.1552 - mean_absolute_error: 261.9455 - mean_absolute_percentage_error: 13.1082 - cosine_proximity: -9.9994e-01
Epoch 1/1
 - 27s - loss: 150285.4729 - mean_squared_error: 150285.4729 - mean_absolute_error: 266.2465 - mean_absolute_percentage_error: 13.3232 - cosine_proximity: -9.9991e-01
Epoch 1/1
 - 27s - loss: 153360.2560 - mean_squared_error: 153360.2560 - mean_absolute_error: 268.0632 - mean_absolute_percentage_error: 13.4144 - cosine_proximity: -9.9928e-01
Epoch 1/1
 - 26s - loss: 141448.8282 - mean_squared_error: 141448.8282 - mean_absolute_error: 257.6277 - mean_absolute_percentage_error: 12.8921 - cosine_proximity: -9.9997e-01
Epoch 1/1
 - 27s - loss: 147693.3252 - mean_squared_error: 147693.3252 - mean_absolute_error: 261.0647 - mean_absol

Epoch 1/1
 - 30s - loss: 152521.1720 - mean_squared_error: 152521.1720 - mean_absolute_error: 267.5055 - mean_absolute_percentage_error: 13.3861 - cosine_proximity: -9.9903e-01
Epoch 1/1
 - 34s - loss: 146692.6197 - mean_squared_error: 146692.6197 - mean_absolute_error: 261.6131 - mean_absolute_percentage_error: 13.0917 - cosine_proximity: -9.9999e-01
Epoch 1/1
 - 30s - loss: 154827.6173 - mean_squared_error: 154827.6173 - mean_absolute_error: 269.8519 - mean_absolute_percentage_error: 13.5037 - cosine_proximity: -9.9909e-01
Epoch 1/1
 - 33s - loss: 159500.0893 - mean_squared_error: 159500.0893 - mean_absolute_error: 274.8700 - mean_absolute_percentage_error: 13.7547 - cosine_proximity: -9.9934e-01
Epoch 1/1
 - 35s - loss: 165283.7282 - mean_squared_error: 165283.7282 - mean_absolute_error: 277.6468 - mean_absolute_percentage_error: 13.8935 - cosine_proximity: -9.9865e-01
Epoch 1/1
 - 33s - loss: 152350.4802 - mean_squared_error: 152350.4802 - mean_absolute_error: 264.5787 - mean_absol

Epoch 1/1
 - 42s - loss: 159888.9423 - mean_squared_error: 159888.9423 - mean_absolute_error: 267.5520 - mean_absolute_percentage_error: 13.3888 - cosine_proximity: -9.9604e-01
Epoch 1/1
 - 40s - loss: 150183.3931 - mean_squared_error: 150183.3931 - mean_absolute_error: 265.8783 - mean_absolute_percentage_error: 13.3048 - cosine_proximity: -9.9952e-01
Epoch 1/1
 - 38s - loss: 150947.8331 - mean_squared_error: 150947.8331 - mean_absolute_error: 264.1921 - mean_absolute_percentage_error: 13.2205 - cosine_proximity: -9.9894e-01
Epoch 1/1
 - 42s - loss: 140582.1041 - mean_squared_error: 140582.1041 - mean_absolute_error: 258.3307 - mean_absolute_percentage_error: 12.9270 - cosine_proximity: -9.9990e-01
Epoch 1/1
 - 42s - loss: 149377.5152 - mean_squared_error: 149377.5152 - mean_absolute_error: 268.0072 - mean_absolute_percentage_error: 13.4116 - cosine_proximity: -9.9963e-01
Epoch 1/1
 - 42s - loss: 153567.4944 - mean_squared_error: 153567.4944 - mean_absolute_error: 269.5645 - mean_absol

In [42]:
def log_errors(y_test, predicted):
    mae = mean_absolute_error(y_test, predicted)
    mse = mean_squared_error(y_test, predicted, multioutput='raw_values')
    r2 = r2_score(y_test, predicted)
    medae = median_absolute_error(y_test, predicted)
    print('mae ',mae)
    print('mse ',mse)
    print('r2 score ', r2)
    print('median_ae ',medae)

In [43]:
estimator.fit(X_train, y_train)
prediction = estimator.predict(X_test)
log_errors(y_test=y_test,predicted=prediction)

/home/phoenix/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=1)`
  


Epoch 1/1
 - 35s - loss: 148569.9657 - mean_squared_error: 148569.9657 - mean_absolute_error: 264.1529 - mean_absolute_percentage_error: 13.2184 - cosine_proximity: -9.9942e-01
mae  155.579564612
mse  [ 44469.7230967]
r2 score  -366.8444787
median_ae  120.43548584
